In [1]:
import os
import glob
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

In [2]:
dataset_directory = '.\\dataset'

path = os.listdir(dataset_directory)

In [3]:
'''
Reading the data using pandas dataframe

'''

labels = pd.read_csv(os.path.join(dataset_directory, 'concepts_2011.txt'), sep="\t")
image_labels = pd.read_csv(os.path.join(dataset_directory, 'trainset_gt_annotations.txt'), sep=" ", header=None)

image_labels[0] = image_labels[0].map(lambda x: x.strip('.jpg'))

list_of_features = [os.path.join(os.path.join(dataset_directory,"imageclef2011_feats") , file) for file in os.listdir(os.path.join(dataset_directory,"imageclef2011_feats")) if file.endswith(".npy") ]

In [4]:
labels

,number,name
0,0,Partylife
1,1,Family_Friends
2,2,Beach_Holidays
3,3,Building_Sights
4,4,Snow
...,...,...
94,94,scary
95,95,unpleasant
96,96,melancholic
97,97,inactive


In [5]:
image_labels

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0039b5a7-c1ad-423a-92a0-3f38558043a2,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,003eaf28-898d-404f-abe5-e8e86d422fa2,0,0,0,0,0,0,0,0,0,...,0,1,1,0,0,0,0,0,1,0
2,005f4848-780a-4d31-8c09-4abdfd46804c,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,1,1,1
3,00a53424-5a0d-4c9d-ac90-242a6099fc35,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
4,00b5c2a5-2b8f-492b-ad5d-ed3310e6da56,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,{FFA81171-399C-432E-8373-753B7FC5F85B},0,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
7996,{FFB02EA2-96FB-4C60-ADEF-C57C0CC2266E},0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
7997,{FFB61D93-7A87-487B-8478-58B78E878823},0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7998,{FFC6F015-ABF9-4AD9-ACC9-0941FD14A251},0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [6]:
list_of_features = [os.path.join(os.path.join(dataset_directory,"imageclef2011_feats") , file) for file in os.listdir(os.path.join(dataset_directory,"imageclef2011_feats")) if file.endswith(".npy") ]

In [7]:
feats = np.array([np.load(feat) for feat in list_of_features])
spring = feats.transpose()[9] #Spring

In [8]:
summer = feats.transpose()[10]#summer

In [9]:
autumn = feats.transpose()[11] #Autumn

In [10]:
winter = feats.transpose()[12] #Winter

In [11]:
x = np.concatenate((spring,summer,autumn,winter))
x

array([0.00030083, 0.00019788, 0.0008226 , ..., 0.00107452, 0.00032849,
       0.00073593], dtype=float32)

In [12]:
x = x.reshape([4,8000]).transpose()

In [13]:
y = np.repeat(np.arange(4),2000)
y.shape

(8000,)

In [26]:
x_train, x_test, y_train, y_test = train_test_split(feats, y, test_size=0.15, stratify=y )
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=2000/6800, stratify=y_train)

In [27]:
x_train.shape, y_train.shape

((4800, 1024), (4800,))

In [28]:
x_test.shape, y_test.shape

((1200, 1024), (1200,))

In [29]:
x_val.shape, y_val.shape

((2000, 1024), (2000,))

In [30]:
regularization_constants = [0.01, 0.1, 0.1**0.5, 1, 10**0.5, 10, 100**0.5]
scores = []

In [ ]:


from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

regularization_constants = [0.01, 0.1, 0.1**0.5, 1, 10**0.5, 10, 100**0.5]
scores = []

for c in regularization_constants:
    svm = OneVsRestClassifier(estimator=SVC(C=c, kernel="linear"))
    svm.fit(x_train, y_train)
    scores.append(svm.score(x_val, y_val))
    
best_c = regularization_constants[np.argmax(scores)]
best_score = np.max(scores)
print("Best pair | c={}, score={}".format(best_c, best_score))

